<a href="https://colab.research.google.com/github/Jinhyeong-Lim/ALCE/blob/main/%5BChapter_2%5D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%BD%94%ED%8D%BC%EC%8A%A4_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 코퍼스 전처리 실습

## Introduction

**Chapter 2. 한국어 데이터 전처리** 강의의 **한국어 코퍼스 전처리 실습 강의**입니다.

일반적인 한국어 코퍼스 처리 과정인 (1) 코퍼스 수집, (2) 정제(Cleaning) 및 정규화(Normalization), (3) 토큰화(Tokenization)을 이해할 수 있도록 간단한 전처리 과정을 실습합니다.

## 1. 코퍼스 수집

한국어 코퍼스 전처리 실습을 위해, 뉴스 기사를 크롤링 해보겠습니다.

일반적으로 크롤링 코드를 작성하려면 많은 요소들을 고려해야하지만,

본 실습에서는 url 정보만 입력해주면 텍스트를 추출해주는 [Newspaper3k](https://github.com/codelucas/newspaper) 라이브러리를 사용하도록 하겠습니다.      

In [ ]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=b42cd2313c3cd5d9c5e6c325a9cf914bd6d94f90e948ba76e7a3212dab088f28
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=cf318f678018e988a8dbab128c26f1f6eff421c63edcd408fa7ba

newspaper 라이브러리는 매우 많은 언어에 대한 크롤링을 지원해주며, 아래 명령어로 지원 가능한 언어 목록을 확인할 수 있습니다.

In [ ]:
import newspaper
newspaper.languages()


Your available languages are:

input code		full name
  ja			  Japanese
  fa			  Persian
  he			  Hebrew
  zh			  Chinese
  no			  Norwegian
  mk			  Macedonian
  sw			  Swahili
  pt			  Portuguese
  uk			  Ukrainian
  ru			  Russian
  hu			  Hungarian
  nl			  Dutch
  es			  Spanish
  be			  Belarusian
  ro			  Romanian
  nb			  Norwegian (Bokmål)
  hi			  Hindi
  sr			  Serbian
  vi			  Vietnamese
  tr			  Turkish
  ko			  Korean
  it			  Italian
  da			  Danish
  ar			  Arabic
  en			  English
  sv			  Swedish
  el			  Greek
  hr			  Croatian
  fi			  Finnish
  de			  German
  sl			  Slovenian
  pl			  Polish
  et			  Estonian
  fr			  French
  bg			  Bulgarian
  id			  Indonesian



이제 다운로드 받은 라이브러리를 활용하여 뉴스를 크롤링해보겠습니다.

일반적인 뉴스 기사의 경우, 상업용으로 사용하거나 재배포하는 등의 활동은 저작권에 위반되기 때문에, 사용 목적에 따라 저작권을 주의 깊게 확인하고 사용해야 합니다.   

본 실습에서는 저작권에서 비교적 자유로운 위키트리 뉴스 데이터를 사용하도록 하겠습니다.   

In [ ]:
from newspaper import Article

In [ ]:
URL = "https://www.wikitree.co.kr/articles/813981"

# title: 온갖 곳에서 다 쓰는 ‘중요한 건 꺾이지 않는 마음’, 사람들 반응은? feat. 중꺾마 [뉴스마켓]
# context: 이하 유튜브 '뉴스마켓'
# 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다. 바로 ‘중요한 것은 꺾이지 않는 마음’인데요.
# 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다. 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
# 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.

In [ ]:
article = Article(URL, language='ko')

In [ ]:
article.download()
article.parse()

print('title:', article.title)
print('context:', article.text)

title: 온갖 곳에서 다 쓰는 ‘중요한 건 꺾이지 않는 마음’, 사람들 반응은? feat. 중꺾마 [뉴스마켓]
context: ※ 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다. '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다. ☞ 유튜브 '뉴스마켓' 구독하기

이하 유튜브 '뉴스마켓'

주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다. 바로 ‘중요한 것은 꺾이지 않는 마음’인데요.

암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다. 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.

희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.


추가로, 다양한 전처리 실습을 위해 일반적인 뉴스에서 많이 포함되는 예제 텍스트들을 임의로 추가할게요.

In [ ]:
additional_info = [
  "영상취재 홍길동(hongkildong@chosun.com) 제주방송 임꺽정 홍길동(hongkildong@chosun.com)",
  "<h1>중요한 것은       꺾이지     않는 마음</h1> <h3>카타르 월드컵</h3> <b>유행어</b>로 이루어진    문서입니다."
  "<br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>",
  "Copyright ⓒ JIBS. All rights reserved. 무단 전재 및 재배포 금지.",
  "<이 기사는 언론사 에서 문화 섹션 으로 분류 했습 니다.>",
  "<br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>",
  "#주가 #부동산 #폭락 #환률 #급상승 #중꺾마"
]

context = article.text.split('\n')
context += additional_info

따라서, 우리가 전처리해야할 대상 코퍼스는 다음과 같습니다.

In [ ]:
for i, text in enumerate(context):
  print(i, text)

0 ※ 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다. '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다. ☞ 유튜브 '뉴스마켓' 구독하기
1 
2 이하 유튜브 '뉴스마켓'
3 
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다. 바로 ‘중요한 것은 꺾이지 않는 마음’인데요.
5 
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다. 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
7 
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동(hongkildong@chosun.com) 제주방송 임꺽정 홍길동(hongkildong@chosun.com)
10 <h1>중요한 것은       꺾이지     않는 마음</h1> <h3>카타르 월드컵</h3> <b>유행어</b>로 이루어진    문서입니다.<br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>
11 Copyright ⓒ JIBS. All rights reserved. 무단 전재 및 재배포 금지.
12 <이 기사는 언론사 에서 문화 섹션 으로 분류 했습 니다.>
13 <br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>
14 #주가 #부동산 #폭락 #환률 #급상승 #중꺾마


## 2. 정제(Cleaning)

정제(Cleaning) 단계에서는 학습에 필요하지 않은 데이터를 제거하거나 필터링하는 일련의 과정들을 진행합니다.

### 2.1. 불용어(stopwords) 제거

영어가 아닌 한국어에서 불용어 제거 작업은 보통, 토큰화 과정과 형태소 분석을 거친 이후 조사나 어미를 제거하는 방식으로 진행됩니다.

하지만, 사전에 추가적으로 제거하고 싶은 불용어들을 사전에 넣어서 별도로 필터링할 수 있습니다.

In [ ]:
# 불용어 사전 정의
stopwords = ['이하', '바로', '☞', '※', '…']

# 불용어 제거
def delete_stopwords(context):
    preprocessed_text = []
    for text in context:
        text = [w for w in text.split(' ') if w not in stopwords]
        preprocessed_text.append(' '.join(text))
    return preprocessed_text

In [ ]:
processed_context = delete_stopwords(context)

In [ ]:
for i, text in enumerate(processed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다. '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다. 유튜브 '뉴스마켓' 구독하기
1 
2 유튜브 '뉴스마켓'
3 
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다. ‘중요한 것은 꺾이지 않는 마음’인데요.
5 
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다. 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
7 
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동(hongkildong@chosun.com) 제주방송 임꺽정 홍길동(hongkildong@chosun.com)
10 <h1>중요한 것은       꺾이지     않는 마음</h1> <h3>카타르 월드컵</h3> <b>유행어</b>로 이루어진    문서입니다.<br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>
11 Copyright ⓒ JIBS. All rights reserved. 무단 전재 및 재배포 금지.
12 <이 기사는 언론사 에서 문화 섹션 으로 분류 했습 니다.>
13 <br>이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…<br>
14 #주가 #부동산 #폭락 #환률 #급상승 #중꺾마


### 2.2. HTML 태그 제거

다음으로 무분별하게 부착되어 있는 HTML 태그들부터 제거해볼게요.

HTML 태그는 웹 크롤링을 하면 많이 딸려오기 때문에 학습에 필요하지 않다면 제거해줘야 합니다.

In [ ]:
import re

In [ ]:
# HTML 태그를 제거하여 반환합니다.
def delete_html_tag(context):
  """
  ex. <h1>뉴스 제목</h1> -> 뉴스 제목
  """
  preprcessed_text = []

  for text in context:
      text = re.sub(r"<[^>]+>\s+(?=<)|<[^>]+>", "", text).strip()
      if text:
          preprcessed_text.append(text)
  return preprcessed_text

In [ ]:
processed_context = delete_html_tag(processed_context)

In [ ]:
for i, text in enumerate(processed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다. '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다. 유튜브 '뉴스마켓' 구독하기
1 유튜브 '뉴스마켓'
2 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다. ‘중요한 것은 꺾이지 않는 마음’인데요.
3 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다. 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
4 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
5 영상취재 홍길동(hongkildong@chosun.com) 제주방송 임꺽정 홍길동(hongkildong@chosun.com)
6 중요한 것은       꺾이지     않는 마음카타르 월드컵유행어로 이루어진    문서입니다.이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
7 Copyright ⓒ JIBS. All rights reserved. 무단 전재 및 재배포 금지.
8 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
9 #주가 #부동산 #폭락 #환률 #급상승 #중꺾마


### 2.3. 문장 분리

다음으로, 한 행에 여러 문장이 있으면, 학습 데이터를 구성할 때 입력 데이터의 단위를 설정하기 애매해지므로,

문장 단위로 모델이 학습하도록 유도하기 위해, 문장 분리를 먼저 해주겠습니다.  

한국어 문장분리기 중, 가장 성능이 우수한 것으로 알려진 [kss 라이브러리](https://github.com/hyunwoongko/kss)를 사용합니다.

In [ ]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.3.1-py3-none-any.whl size=52145 sha256=52a8a17e2e08e590607f3874d038d6061731479f85a133ecf30169053820bbe8
  Stored in directory: /root/.cache/pip/wheels/40/6e/54/257ecc4412c109490a93e8d3e729aab9d78bde856b3e969e67
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646667 sha256=2fa8d5f7cc0c5a0d4218f52ac436200b58a1949c8bd8a43840d5df7fc61e082b
  Stored in directory: /root/.cache/pip/wheels/a9/98/a3/085a909aa25805f321467a3af3b80d43b6094ed335b4818775
Successfully built kss pecab


In [ ]:
import kss

In [ ]:
# 한 줄 단위로 문장 분리를 진행
def sentence_seperator(processed_context):
  splited_context = []

  for text in processed_context:
      text = text.strip()
      if text:
          splited_text = kss.split_sentences(text)
          splited_context.extend(splited_text)

  return splited_context

In [ ]:
splited_context = sentence_seperator(processed_context)

In [ ]:
for i, sent in enumerate(splited_context):
    print(i, sent)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동(hongkildong@chosun.com) 제주방송 임꺽정 홍길동(hongkildong@chosun.com)
10 중요한 것은       꺾이지     않는 마음카타르 월드컵유행어로 이루어진    문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 Copyright ⓒ JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
16 #주가 #부동산 #폭락 #환률 #급상승 #중꺾마


### 2.4. 이메일 제거

In [ ]:
# 코퍼스 내 이메일을 제거합니다.
def delete_email(context):
  """
  ex. 홍길동(hongkildong@chosun.com) -> 홍길동()
  """
  preprocessed_text = []
  for text in context:
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "", text).strip()
    if text:
      preprocessed_text.append(text)
  return preprocessed_text

In [ ]:
preprocessed_context = delete_email(splited_context)

In [ ]:
for i, text in enumerate(preprocessed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동() 제주방송 임꺽정 홍길동()
10 중요한 것은       꺾이지     않는 마음카타르 월드컵유행어로 이루어진    문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 Copyright ⓒ JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
16 #주가 #부동산 #폭락 #환률 #급상승 #중꺾마


### 2.5. 해시태그 제거

In [ ]:
# 코퍼스 내 해시태그(#)를 제거합니다.
def delete_hashtag(context):
  """
  ex. #주가 #부동산 -> 주가 부동산
  """
  preprocessed_text = []
  for text in context:
    text = re.sub(r"#\S+", "", text).strip()
    if text:
      preprocessed_text.append(text)
  return preprocessed_text

In [ ]:
preprocessed_context = delete_hashtag(preprocessed_context)

In [ ]:
for i, text in enumerate(preprocessed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동() 제주방송 임꺽정 홍길동()
10 중요한 것은       꺾이지     않는 마음카타르 월드컵유행어로 이루어진    문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 Copyright ⓒ JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 2.6. 기타 학습에 필요 없는 데이터 제거

In [ ]:
# 뉴스 내 포함된 저작권 관련 텍스트를 제거
def delete_copyright(context):
  re_patterns = [
    r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
    r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
  ]
  preprocessed_text = []
  for text in context:
    for re_pattern in re_patterns:
      text = re.sub(re_pattern, "", text).strip()
    if text:
      preprocessed_text.append(text)
  return preprocessed_text

In [ ]:
preprocessed_context = delete_copyright(preprocessed_context)

In [ ]:
for i, text in enumerate(preprocessed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동() 제주방송 임꺽정 홍길동()
10 중요한 것은       꺾이지     않는 마음카타르 월드컵유행어로 이루어진    문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 실습1. 한국어 데이터 정제(cleaning) 실습

추가로, 정제가 필요한 항목이 있다면, 여러분이 텍스트를 정제하고 확인해보세요!

In [ ]:
# Free Trial 1












## 3. 정규화 (Normalizing)

정규화(Normalizing) 단계는 코퍼스 내에 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어주는 단계입니다.

정규화 과정에는 [Soynlp](https://github.com/lovit/soynlp)를 활용해보겠습니다.

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 8.6 MB/s eta 0:00:00


### 3.1. 반복 횟수가 너무 많은 문자 정규화

In [ ]:
from soynlp.normalizer import *

In [ ]:
print(repeat_normalize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', num_repeats=2))
print(repeat_normalize('얔ㅋㅋㅋㅋㅋㅋㅋ 너 ㅋㅋㅋㅋ 지금 뭐하냐 ㅠㅠㅠㅠ', num_repeats=2))

ㅋㅋ
얔ㅋㅋ 너 ㅋㅋ 지금 뭐하냐 ㅠㅠ


In [ ]:
# 반복되는 문자 정규화
def repeat_char_nomalizer(context):
  normalized_text = []
  for text in context:
    text = repeat_normalize(text, num_repeats=2).strip()
    if text:
      normalized_text.append(text)
  return normalized_text

In [ ]:
normalized_context = repeat_char_nomalizer(preprocessed_context)

In [ ]:
for i, text in enumerate(normalized_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동() 제주방송 임꺽정 홍길동()
10 중요한 것은 꺾이지 않는 마음카타르 월드컵유행어로 이루어진 문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 3.2. 띄어쓰기 정규화

In [ ]:
# 두 개 이상의 연속된 공백을 하나로 치환합니다.
def repeated_spacing_nomalizer(context):
  normalized_text = []
  for text in context:
    text = re.sub(r"\s+", " ", text).strip()
    if text:
        normalized_text.append(text)
  return normalized_text

In [ ]:
normalized_context = repeated_spacing_nomalizer(normalized_context)

In [ ]:
for i, text in enumerate(normalized_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스마켓'을 통해 독자 여러분과 만납니다.
1 '뉴스마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스마켓' 구독하기
3 유튜브 '뉴스마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상취재 홍길동() 제주방송 임꺽정 홍길동()
10 중요한 것은 꺾이지 않는 마음카타르 월드컵유행어로 이루어진 문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 3.3. 띄어쓰기 보정

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-enqpcdjy
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-enqpcdjy
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 1f8d11c59ac93525432f164e4c237ece4e298691
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.4 MB/s eta 0:00:00
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268637 sha256=e1868a173dbf428f64e1123d9e1d3fa255d71e612606f3124dc8a93898d1d323
  Stored in directory: /tmp/pip-ephem-wheel-cache-wi78vssj/wheels/79/a0/33/16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully

In [ ]:
from pykospacing import Spacing

In [ ]:
spacing = Spacing()
spacing("아버지가방에들어가신다")

1/1 [==============================] - 1s 755ms/step


'아버지가 방에 들어가신다'

In [ ]:
spacing = Spacing()
spacing("아버 지가방에들 어가신다")

1/1 [==============================] - 0s 47ms/step


'아버 지가 방에 들 어가신다'

In [ ]:
# 잘못된 띄어쓰기를 보정
def spacing_sent(context):
  normalized_text = []
  for text in context:
    text = spacing(text)
    if text:
      normalized_text.append(text)
  return normalized_text

In [ ]:
normalized_context = spacing_sent(normalized_context)

1/1 [==============================] - 0s 43ms/step


In [ ]:
for i, text in enumerate(normalized_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스 마켓'을 통해 독자 여러분과 만 납니다.
1 '뉴스 마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스 마켓' 구독하기
3 유튜브 '뉴스 마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로 잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어 나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상 취재 홍길동() 제 주방송 임꺽정 홍길동()
10 중요한 것은 꺾이지 않는 마음카타르 월드컵 유행어로 이루어진 문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.
15 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 3.4. 중복 문장 정규화

In [ ]:
from collections import OrderedDict

In [ ]:
# 중복된 문장을 제거합니다.
def duplicated_sentence_nomalizer(context):
    context = list(OrderedDict.fromkeys(context))
    return context

In [ ]:
normalized_context = duplicated_sentence_nomalizer(normalized_context)

In [ ]:
for i, text in enumerate(normalized_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스 마켓'을 통해 독자 여러분과 만 납니다.
1 '뉴스 마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 유튜브 '뉴스 마켓' 구독하기
3 유튜브 '뉴스 마켓'
4 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로 잡았습니다.
5 ‘중요한 것은 꺾이지 않는 마음’인데요.
6 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어 나갔니다.
7 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
8 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
9 영상 취재 홍길동() 제 주방송 임꺽정 홍길동()
10 중요한 것은 꺾이지 않는 마음카타르 월드컵 유행어로 이루어진 문서입니다.
11 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…
12 JIBS.
13 All rights reserved.
14 무단 전재 및 재배포 금지.


## 4. 남은 전처리 (Cleaning)

### 4.1. 형태소 분석 기반 필터링

마지막으로, 형태소 분석 기반 필터링을 사용하여 정규화된 데이터를 후처리합니다.


한국어에서는 조사나 어미가 불용어일 확률이 높기 때문에, 형태소 분석기를 활용하여 조사나 어미와 같은 불용어를 제거하곤 합니다.

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 27.7 MB/s eta 0:00:00


In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [ ]:
from konlpy.tag import Mecab

In [ ]:
mecab = Mecab()
morphs = mecab.pos("아버지가방에들어가신다.", join=False)
print(morphs)

[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV'), ('신다', 'EP+EF'), ('.', 'SF')]


In [ ]:
# 명사(NN), 동사(V), 형용사(J)의 포함 여부에 따라 문장 필터링
def morph_filter(context):
  NN_TAGS = ["NNG", "NNP", "NNB", "NP"]
  V_TAGS = ["VV", "VA", "VX", "VCP", "VCN", "XSN", "XSA", "XSV"]
  J_TAGS = ["JKS", "J", "JO", "JK", "JKC", "JKG", "JKB", "JKV", "JKQ", "JX", "JC", "JKI", "JKO", "JKM", "ETM"]

  preprocessed_text = []

  for text in context:
    morphs = mecab.pos(text, join=False)

    nn_flag = False
    v_flag = False
    j_flag = False
    for morph in morphs:
      pos_tags = morph[1].split("+")
      for pos_tag in pos_tags:
        if not nn_flag and pos_tag in NN_TAGS:
          nn_flag = True
        if not v_flag and pos_tag in V_TAGS:
          v_flag = True
        if not j_flag and pos_tag in J_TAGS:
          j_flag = True
      if nn_flag and v_flag and j_flag:
        preprocessed_text.append(text)
        break
  return preprocessed_text

In [ ]:
post_processed_context = morph_filter(normalized_context)

In [ ]:
for i, text in enumerate(post_processed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스 마켓'을 통해 독자 여러분과 만 납니다.
1 '뉴스 마켓'은 쉽고 빠르게 실시간 뉴스를 영상으로 접할 수 있는 채널로, 앞으로 더 알차고 좋은 내용으로 꾸려나갈 예정입니다.
2 주가, 부동산 폭락, 환율 급상승 등 우울한 일로 가득했던 2022년, 월드컵에서 등장한 이 문장이 대한민국을 사로 잡았습니다.
3 ‘중요한 것은 꺾이지 않는 마음’인데요.
4 암울한 현실 상황으로 인해 한 해 동안 유행어는 대부분 냉소적인 내용들로 이루어져 있었지만, ‘중요한 건 꺾이지 않는 마음’의 긍정적인 의미는 인터넷을 넘어 방송, 정치 등으로 뻗어 나갔니다.
5 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
6 희망과 노력의 가치를 일깨워 준 유행어 "꺾이지 않는 마음"은 모두가 힘들었던 2022년 겨울, 뜨거웠던 카타르 월드컵의 추억을 앞으로도 계속 떠올리게 해줄 것 같습니다.
7 중요한 것은 꺾이지 않는 마음카타르 월드컵 유행어로 이루어진 문서입니다.
8 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


### 4.2. 문장 길이 기반 필터링

In [ ]:
# 문장을 최대, 최소 길이로 필터링합니다.
def min_max_filter(min_len, max_len, context):
  preprocessed_text = []
  for text in context:
    if min_len < len(text) and len(text) < max_len:
      preprocessed_text.append(text)
  return preprocessed_text

In [ ]:
post_processed_context = min_max_filter(20, 60, post_processed_context)

In [ ]:
for i, text in enumerate(post_processed_context):
    print(i, text)

0 위키트리가 선보이는 영상 뉴스가 유튜브 '뉴스 마켓'을 통해 독자 여러분과 만 납니다.
1 ‘중요한 것은 꺾이지 않는 마음’인데요.
2 일부에서는 너무 남용되어 금방 힘이 꺾일 것이라는 전망도 나오는데요.
3 중요한 것은 꺾이지 않는 마음카타르 월드컵 유행어로 이루어진 문서입니다.
4 이 줄은 실제 뉴스에 포함되지 않은 임시 데이터임을 알립니다…


## 5. 토큰화(Tokenization)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(post_processed_context)

In [ ]:
word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(post_processed_context)

In [ ]:
vocab_size = len(word2idx) + 1
print(f"단어 사전의 크기 : {vocab_size}")

단어 사전의 크기 : 42


In [ ]:
print(encoded[0])

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [ ]:
word2idx

{'것은': 1,
 '꺾이지': 2,
 '않는': 3,
 '위키트리가': 4,
 '선보이는': 5,
 '영상': 6,
 '뉴스가': 7,
 '유튜브': 8,
 "'뉴스": 9,
 "마켓'을": 10,
 '통해': 11,
 '독자': 12,
 '여러분과': 13,
 '만': 14,
 '납니다': 15,
 '‘중요한': 16,
 '마음’인데요': 17,
 '일부에서는': 18,
 '너무': 19,
 '남용되어': 20,
 '금방': 21,
 '힘이': 22,
 '꺾일': 23,
 '것이라는': 24,
 '전망도': 25,
 '나오는데요': 26,
 '중요한': 27,
 '마음카타르': 28,
 '월드컵': 29,
 '유행어로': 30,
 '이루어진': 31,
 '문서입니다': 32,
 '이': 33,
 '줄은': 34,
 '실제': 35,
 '뉴스에': 36,
 '포함되지': 37,
 '않은': 38,
 '임시': 39,
 '데이터임을': 40,
 '알립니다…': 41}